# Scraping and Data Cleaning

We begin the project with scraping and data cleaning, as the data will be needed for future steps such as modelling (02-modelling) and exploratory data analyses (03-eda).

### Contents

- [Scraping](#Scraping)
    - [Import Libraries](#Import-Libraries)
    - [Scraping Club Links](#Scraping-Club-Links)
    - [Scraping Player Links](#Scraping-Player-Links)
    - [Retrieve player statistics from Player Page](#Retrieve-player-statistics-from-Player-Page)
- [Data Cleaning](#Data-Cleaning)
    - [Drop DOB Columns](#Drop-DOB-Columns)
    - [Focus on key stats](#Focus-on-key-stats)
    - [Shift player information to the front](#Shift-player-information-to-the-front)
    - [Retain Percentage Data](#Retain-Percentage-Data)
    - [Remove Foreigners](#Remove-Foreigners)
    - [Rescoring columns to mitigate 0 values](#Rescoring-columns-to-mitigate-0-values)
    - [Rescoring keeper stat columns](#Rescoring-keeper-stat-columns)
    - [Generating Role Scores](#Generating-Role-Scores)    
    - [Generating Recommendation Scores](#Generating-Recommendation-Scores)
- [Next Steps](#Next-Steps)
    - [Modelling](#Modelling)
    - [EDA](#EDA)

## Scraping

### Import Libraries

In [1]:
#Import libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import requests

### Scraping Club Links

This code is aimed at obtaining the links to the local clubs, which will then be fed into the following code.

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Initialize the WebDriver (Make sure you have the appropriate driver installed, e.g., chromedriver for Chrome)
driver = webdriver.Chrome()

# Navigate to the webpage
driver.get('https://www.sofascore.com/tournament/football/singapore/premier-league/634')

# Find all div elements with the specified class name using a CSS selector
div_elements = driver.find_elements(By.CSS_SELECTOR, "div.Box.eHXJll")

# Create a list to store all the player page urls
team_urls = []

# Loop through each div element and find all a elements within it
for div in div_elements:
    # Find all <a> elements within the current <div>
    link_elements = div.find_elements(By.CSS_SELECTOR, "a")
    
    # Loop through each <a> element and print its href attribute
    for link in link_elements:
        href = link.get_attribute('href')
        team_urls.append(href)
        print(href)

# Close the WebDriver
driver.quit()

https://www.sofascore.com/team/football/tampines-rovers/3043
https://www.sofascore.com/team/football/lion-city-sailors/3041
https://www.sofascore.com/team/football/geylang-international/3044
https://www.sofascore.com/team/football/balestier-central/3039
https://www.sofascore.com/team/football/hougang-united/34083
https://www.sofascore.com/team/football/brunei-dpmm-fc/34085
https://www.sofascore.com/team/football/albirex-niigata-singapore/34081
https://www.sofascore.com/team/football/young-lions/34082
https://www.sofascore.com/team/football/tanjong-pagar-united/3038


In [96]:
# Check number of club links obtained
len(team_urls)

9

### Scraping Player Links

This code is aimed at obtaining the links to the players at the aforementioned clubs, which will then be fed into the following code.

In [5]:
driver = webdriver.Chrome()

# Create a list to store all the player page urls
player_urls = []

for team_url in team_urls:
    
    # Navigate to the webpage
    driver.get(team_url)

    # Find the element using XPath and click it
    element = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[1]/div[3]/div[1]/div/div/div/h2[4]/a")
    element.click()

    # Find all div elements with the specified class name using a CSS selector
    div_elements = driver.find_elements(By.CSS_SELECTOR, "div.Box.dflyPx")

    # Loop through each div element and find all a elements within it
    for div in div_elements:
        # Find all <a> elements within the current <div>
        link_elements = div.find_elements(By.CSS_SELECTOR, "a")
    
        # Loop through each <a> element and print its href attribute
        for link in link_elements:
            href = link.get_attribute('href')
            player_urls.append(href)
            print(href)

# Close the WebDriver
driver.quit()

https://www.sofascore.com/player/boris-kopitovic/843260
https://www.sofascore.com/player/kunori-seia/1218518
https://www.sofascore.com/player/thitipat-ekarunpong/1519378
https://www.sofascore.com/player/faris-ramli/851803
https://www.sofascore.com/player/kyoga-nakamura/358968
https://www.sofascore.com/player/shah-shahiran/929540
https://www.sofascore.com/player/taufik-suparno/851810
https://www.sofascore.com/player/joel-chew-joon-herng/979261
https://www.sofascore.com/player/yasir-hanapi/812590
https://www.sofascore.com/player/saifullah-akbar/835102
https://www.sofascore.com/player/caelan-cheong-tze-jay/1401682
https://www.sofascore.com/player/kieran-tan/1860320
https://www.sofascore.com/player/glenn-kweh/1116213
https://www.sofascore.com/player/shuya-yamashita/1108290
https://www.sofascore.com/player/jared-gallagher/1184545
https://www.sofascore.com/player/milos-zlatkovic/795891
https://www.sofascore.com/player/irfan-najeeb/922337
https://www.sofascore.com/player/amirul-adli-bin-azmi/

In [7]:
# Check number of players available
len(player_urls)

234

### Retrieve player statistics from Player Page

This code will pull player statistics. We will filter for players who only have player data for 2024 Singapore Premier League, to ensure the players that eventually get recommended are actually getting game time in 2024.

We then retrieve the key player statistics, which will be key to our models for each position, as well as basic information such as player position, nationality so we can make use of it later on for filtering or position assignment.

In [6]:
# Initialize the WebDriver (Make sure you have the appropriate driver installed, e.g., chromedriver for Chrome)
driver = webdriver.Chrome()

# Instantiate a list to store the stats of each player
players_list = []

for player_url in player_urls:
    
    # Navigate to the webpage
    driver.get(player_url)

    # Find the element using XPath and click it
    #element = driver.find_element(By.XPATH, "/html/body/div[1]/main/div[1]/div[2]/div[1]/div/div/div/h2[2]/a")
    #element.click()

    # Pause to allow the page to load with a random delay
    time.sleep(random.uniform(2, 3))
    
    # Check if the bdi element contains the value 2024
    try:
        bdi_element = driver.find_element(By.CSS_SELECTOR, "bdi.Text.jFxLbA")
        if bdi_element.text != "2024":
            continue  # Skip this player if the value is not 2024
    except:
        continue  # Skip this player if the bdi element is not found
    
    # Define the CSS selectors
    css_selectors = [
        "div.Box.Flex.dlyXLO.bnpRyo",
        "div.Box.fGLgkO",
        "div.Box.jwDcoO"
    ]
    
    player_list = []

    # Find and collect elements for each CSS selector
    for selector in css_selectors:
        div_elements = driver.find_elements(By.CSS_SELECTOR, selector)
        for div in div_elements:
            lines = div.text.split("\n")
            if len(lines) >= 2:  # Ensure there are at least two lines to avoid errors
                player_list.append(lines[0])
                player_list.append(lines[1])
    
    # Initialize an empty dictionary
    player_dict = {}

    # Extract the string between the last two backslashes
    last_part = player_url.rstrip('/').split('/')[-2]

    # Remove any hyphens from the extracted string
    last_part_cleaned = last_part.replace('-', ' ')

    # Store player name
    player_dict["Player Name"] = last_part_cleaned

    # Loop through the list with index
    for i in range(0, len(player_list), 2):
        # Assign even-indexed value as key and odd-indexed value as value
        player_dict[player_list[i]] = player_list[i + 1]

    # Print the resulting dictionary
    players_list.append(player_dict)
    
driver.quit()

# Create and display data frame of players
players_df = pd.DataFrame(players_list)
players_df

,Player Name,Total played,Started,Minutes per game,Goals,Scoring frequency,Goals per game,Shots per game,Shots on target per game,Goal conversion,...,17 DEC 2001,23 NOV 2004,18 MAY 1995,14 FEB 1995,30 MAY 1995,28 APR 2000,5 DEC 2006,15 FEB 1995,21 AUG 2001,14 NOV 2000
0,boris kopitovic,13,13,90,12,101 min,0.9,2.9,1.5,32%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,kunori seia,13,13,90,10,119 min,0.8,3.5,1.7,22%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,thitipat ekarunpong,1,0,10,0,NaN,0.0,0.0,0.0,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,faris ramli,13,12,82,4,265 min,0.3,2.2,0.9,14%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,kyoga nakamura,13,13,90,0,NaN,0.0,2.2,0.5,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,syed akmal,12,10,81,0,NaN,0.0,0.3,0.1,0%,...,NaN,NaN,NaN,NaN,NaN,24 yrs,NaN,NaN,NaN,NaN
178,seddon casey,1,0,51,0,NaN,0.0,0.0,0.0,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,17 yrs,NaN,NaN,NaN
179,fashah iskandar,4,4,90,0,NaN,0.0,0.0,0.0,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29 yrs,NaN,NaN
180,prathip ekamparam,7,6,90,0,NaN,0.0,0.0,0.0,0%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23 yrs,NaN


We export the data, as this can then potentially be used for the analysis should the end user not want to scrape for the data when needed.

In [91]:
# Export extracted data to csv file
players_df.to_csv('players_raw.csv', index=False)

## Data Cleaning

The below loading is included to ensure we can carry out the data cleaning without having to scrape the website again.

In [2]:
# Load the dataset
file_path = '../data/players_raw.csv'
players_df = pd.read_csv(file_path)

With the raw player information, we now begin with data cleaning. Do note that all data cleaning steps will be executed on the Streamlit app, as the app is designed to scrape, clean and then display predictions and generate squads.

### Drop DOB Columns
The first step we do is to keep the columns we need, and drop all other columns. The data scraping has led to over 200 columns, due to how the website is structured; date of births are one of the column headers, and as such we have essentially one column for every player.

As such, we drop all the date of birth columns in this step.

We display the processed data to ensure that the correct columns are kept.

In [3]:
#Drop Junk Columns
# List of columns to keep based on index (0-based index)
columns_to_keep = list(range(0, 32)) + [33, 34, 35, 36, 48] + list(range(58, 63))

# Select columns
players_df_filtered = players_df.iloc[:, columns_to_keep]

players_df_filtered

,Player Name,Total played,Started,Minutes per game,Goals,Scoring frequency,Goals per game,Shots per game,Shots on target per game,Goal conversion,...,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Clean sheets,Goals conceded per game,Saves per game,Goals conceded,Saves made,Goal kicks per game
0,boris kopitovic,13,13,90,12,101 min,0.9,2.9,1.5,32%,...,189 cm,Both,9.0,F,NaN,NaN,NaN,NaN,NaN,NaN
1,kunori seia,13,13,90,10,119 min,0.8,3.5,1.7,22%,...,180 cm,Right,7.0,F,NaN,NaN,NaN,NaN,NaN,NaN
2,thitipat ekarunpong,1,0,10,0,NaN,0.0,0.0,0.0,0%,...,183 cm,Left,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN
3,faris ramli,13,12,82,4,265 min,0.3,2.2,0.9,14%,...,168 cm,Right,30.0,M,NaN,NaN,NaN,NaN,NaN,NaN
4,kyoga nakamura,13,13,90,0,NaN,0.0,2.2,0.5,0%,...,166 cm,Left,10.0,M,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,syed akmal,12,10,81,0,NaN,0.0,0.3,0.1,0%,...,NaN,NaN,5.0,D,0.0,NaN,NaN,NaN,NaN,NaN
178,seddon casey,1,0,51,0,NaN,0.0,0.0,0.0,0%,...,178 cm,NaN,54.0,D,0.0,NaN,NaN,NaN,NaN,NaN
179,fashah iskandar,4,4,90,0,NaN,0.0,0.0,0.0,0%,...,187 cm,Right,18.0,G,0.0,3.0,4.5 (60%),12.0,18.0,12.0
180,prathip ekamparam,7,6,90,0,NaN,0.0,0.0,0.0,0%,...,187 cm,Right,1.0,G,0.0,2.6,4.4 (63%),18.0,31.0,9.4


### Focus on key stats

Although now we are left with all the player stats, we will focus on just 14 key stats, as such we keep those columns plus the player information (position, nationality, etc.)

Again, we print it out to ensure the correct columns are kept.

In [4]:
# Define new columns to keep for further analysis
new_columns_to_keep = [0,6,12] + list(range(14, 21)) + list(range(22, 25)) + list(range(31,37)) + [38]

# Filter the DataFrame again
players_df_analysis = players_df_filtered.iloc[:, new_columns_to_keep]

# Print or save the filtered DataFrame for analysis
players_df_analysis

,Player Name,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Clean sheets,Saves per game
0,boris kopitovic,0.9,2,12.8 (80%),0.3 (67%),0.5 (33%),1.4,1.3,0.2,0.4,0.9 (92%),4.3 (25%),0.0 (0%),MNE,189 cm,Both,9.0,F,NaN,NaN
1,kunori seia,0.8,4,22.2 (75%),1.3 (55%),1.2 (28%),1.9,3.0,1.2,0.5,2.2 (68%),4.8 (33%),0.5 (29%),JPN,180 cm,Right,7.0,F,NaN,NaN
2,thitipat ekarunpong,0.0,0,3.0 (75%),0.0 (0%),0.0 (0%),0.0,0.0,0.0,0.0,0.0 (0%),0.0 (0%),0.0 (0%),THA,183 cm,Left,NaN,F,NaN,NaN
3,faris ramli,0.3,5,20.7 (71%),0.5 (39%),2.1 (39%),1.3,3.3,1.2,0.2,4.1 (82%),6.4 (48%),0.2 (30%),SIN,168 cm,Right,30.0,M,NaN,NaN
4,kyoga nakamura,0.0,3,59.1 (87%),4.3 (66%),0.8 (46%),3.2,6.2,2.1,1.2,2.2 (88%),5.8 (44%),0.4 (56%),JPN,166 cm,Left,10.0,M,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,syed akmal,0.0,0,29.8 (86%),3.6 (52%),0.1 (25%),4.2,7.6,1.7,2.1,0.0 (0%),2.3 (31%),0.9 (55%),SIN,NaN,NaN,5.0,D,0.0,NaN
178,seddon casey,0.0,0,9.0 (69%),0.0 (0%),0.0 (0%),0.0,3.0,2.0,0.0,1.0 (100%),8.0 (53%),1.0 (33%),ENG,178 cm,NaN,54.0,D,0.0,NaN
179,fashah iskandar,0.0,0,11.0 (61%),3.3 (36%),0.0 (0%),4.5,7.0,0.0,1.0,0.0 (0%),0.5 (67%),0.3 (100%),SIN,187 cm,Right,18.0,G,0.0,4.5 (60%)
180,prathip ekamparam,0.0,0,14.1 (85%),1.0 (35%),0.0 (0%),2.7,4.9,0.3,0.9,0.0 (0%),1.0 (88%),0.7 (100%),SIN,187 cm,Right,1.0,G,0.0,4.4 (63%)


### Shift player information to the front
Next up, to make any analysis later easier, we shift the player information to the front columns so that all player stats are together.

Again, we print it to make sure the columns have indeed been shifted.

In [5]:
# Define the original order of columns
original_columns = players_df_analysis.columns.tolist()

# Define the indices of columns to move and their new positions
columns_to_move = original_columns[13:18]  # Columns 13 to 17
remaining_columns = [col for col in original_columns if col not in columns_to_move]

# Define the new column order
new_order = remaining_columns[:1] + columns_to_move + remaining_columns[1:]

# Rearrange columns
players_df_rearranged = players_df_analysis[new_order]

players_df_rearranged

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,Clean sheets,Saves per game
0,boris kopitovic,MNE,189 cm,Both,9.0,F,0.9,2,12.8 (80%),0.3 (67%),0.5 (33%),1.4,1.3,0.2,0.4,0.9 (92%),4.3 (25%),0.0 (0%),NaN,NaN
1,kunori seia,JPN,180 cm,Right,7.0,F,0.8,4,22.2 (75%),1.3 (55%),1.2 (28%),1.9,3.0,1.2,0.5,2.2 (68%),4.8 (33%),0.5 (29%),NaN,NaN
2,thitipat ekarunpong,THA,183 cm,Left,NaN,F,0.0,0,3.0 (75%),0.0 (0%),0.0 (0%),0.0,0.0,0.0,0.0,0.0 (0%),0.0 (0%),0.0 (0%),NaN,NaN
3,faris ramli,SIN,168 cm,Right,30.0,M,0.3,5,20.7 (71%),0.5 (39%),2.1 (39%),1.3,3.3,1.2,0.2,4.1 (82%),6.4 (48%),0.2 (30%),NaN,NaN
4,kyoga nakamura,JPN,166 cm,Left,10.0,M,0.0,3,59.1 (87%),4.3 (66%),0.8 (46%),3.2,6.2,2.1,1.2,2.2 (88%),5.8 (44%),0.4 (56%),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,syed akmal,SIN,NaN,NaN,5.0,D,0.0,0,29.8 (86%),3.6 (52%),0.1 (25%),4.2,7.6,1.7,2.1,0.0 (0%),2.3 (31%),0.9 (55%),0.0,NaN
178,seddon casey,ENG,178 cm,NaN,54.0,D,0.0,0,9.0 (69%),0.0 (0%),0.0 (0%),0.0,3.0,2.0,0.0,1.0 (100%),8.0 (53%),1.0 (33%),0.0,NaN
179,fashah iskandar,SIN,187 cm,Right,18.0,G,0.0,0,11.0 (61%),3.3 (36%),0.0 (0%),4.5,7.0,0.0,1.0,0.0 (0%),0.5 (67%),0.3 (100%),0.0,4.5 (60%)
180,prathip ekamparam,SIN,187 cm,Right,1.0,G,0.0,0,14.1 (85%),1.0 (35%),0.0 (0%),2.7,4.9,0.3,0.9,0.0 (0%),1.0 (88%),0.7 (100%),0.0,4.4 (63%)


### Retain Percentage Data

Some columns have both a flat value together with percentages. We keep the percentages as we want to look at their performance on a particular skill, and how consistently they can produce that skill. These include winning aerial duels, successful passes. Retaining the percentages will allow us to look at their average performance, whereas looking at the pure numbers may skew due to one off good performance in certain games.

In [6]:
# Define the columns with percentage data
percentage_columns = list(range(8, 11)) + list(range(15, 18)) + [19]

# Function to extract percentage and convert to decimal
def extract_percentage(value):
    if isinstance(value, str):
        match = re.search(r'\((\d+)%\)', value)
        if match:
            return float(match.group(1)) / 100
    return None

# Apply the function to each relevant column
for col_index in percentage_columns:
    col_name = players_df_rearranged.columns[col_index]
    # Convert values to string and apply the function
    players_df_rearranged.loc[:, col_name] = players_df_rearranged[col_name].astype(str).apply(extract_percentage)

players_df_rearranged

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,Clean sheets,Saves per game
0,boris kopitovic,MNE,189 cm,Both,9.0,F,0.9,2,0.8,0.67,0.33,1.4,1.3,0.2,0.4,0.92,0.25,0.0,NaN,NaN
1,kunori seia,JPN,180 cm,Right,7.0,F,0.8,4,0.75,0.55,0.28,1.9,3.0,1.2,0.5,0.68,0.33,0.29,NaN,NaN
2,thitipat ekarunpong,THA,183 cm,Left,NaN,F,0.0,0,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,faris ramli,SIN,168 cm,Right,30.0,M,0.3,5,0.71,0.39,0.39,1.3,3.3,1.2,0.2,0.82,0.48,0.3,NaN,NaN
4,kyoga nakamura,JPN,166 cm,Left,10.0,M,0.0,3,0.87,0.66,0.46,3.2,6.2,2.1,1.2,0.88,0.44,0.56,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,syed akmal,SIN,NaN,NaN,5.0,D,0.0,0,0.86,0.52,0.25,4.2,7.6,1.7,2.1,0.0,0.31,0.55,0.0,NaN
178,seddon casey,ENG,178 cm,NaN,54.0,D,0.0,0,0.69,0.0,0.0,0.0,3.0,2.0,0.0,1.0,0.53,0.33,0.0,NaN
179,fashah iskandar,SIN,187 cm,Right,18.0,G,0.0,0,0.61,0.36,0.0,4.5,7.0,0.0,1.0,0.0,0.67,1.0,0.0,0.6
180,prathip ekamparam,SIN,187 cm,Right,1.0,G,0.0,0,0.85,0.35,0.0,2.7,4.9,0.3,0.9,0.0,0.88,1.0,0.0,0.63


### Remove Foreigners

Since the aim of this app is to select for the Singapore national team, foreign players have to be excluded.

We print it out to ensure all foreigners have been removed.

In [7]:
# Filter to retain only players with SIN nationality
players_df_sin = players_df_rearranged[players_df_rearranged.iloc[:, 1] == 'SIN']

# Display the filtered DataFrame
players_df_sin

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,Clean sheets,Saves per game
3,faris ramli,SIN,168 cm,Right,30.0,M,0.3,5,0.71,0.39,0.39,1.3,3.3,1.2,0.2,0.82,0.48,0.3,NaN,NaN
5,shah shahiran,SIN,173 cm,Right,8.0,M,0.0,2,0.92,0.77,0.3,3.6,6.8,1.9,0.5,0.91,0.34,0.14,NaN,NaN
6,taufik suparno,SIN,165 cm,Right,13.0,M,0.2,2,0.77,1.0,0.27,0.9,1.5,0.4,0.5,0.62,0.42,0.58,NaN,NaN
7,joel chew joon herng,SIN,168 cm,Right,12.0,M,0.0,0,0.82,0.0,0.0,0.5,0.0,0.0,0.5,0.0,0.0,0.0,NaN,NaN
8,yasir hanapi,SIN,170 cm,Right,18.0,M,0.1,3,0.9,0.83,0.7,0.6,0.8,0.3,0.3,0.67,0.28,0.57,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,faizal roslan,SIN,181 cm,Right,15.0,D,0.1,0,0.79,0.56,0.06,5.1,7.7,3.2,2.5,1.0,0.36,0.53,0.0,NaN
177,syed akmal,SIN,NaN,NaN,5.0,D,0.0,0,0.86,0.52,0.25,4.2,7.6,1.7,2.1,0.0,0.31,0.55,0.0,NaN
179,fashah iskandar,SIN,187 cm,Right,18.0,G,0.0,0,0.61,0.36,0.0,4.5,7.0,0.0,1.0,0.0,0.67,1.0,0.0,0.6
180,prathip ekamparam,SIN,187 cm,Right,1.0,G,0.0,0,0.85,0.35,0.0,2.7,4.9,0.3,0.9,0.0,0.88,1.0,0.0,0.63


We extract the file to CSV as data cleaning has been completed, and this version of the file will be used for some exploratory data analyses - found in (01-EDA.ipynb).

In [ ]:
players_df_sin.to_csv('players_df_sin_eda.csv', index=False)

### Rescoring columns to mitigate 0 values

As some player stats are 0 and the aim of the app is to compare between players. Relative performance becomes important. As such, we rescore the columns to have a base of 25, so that the formulas for the role scores will not be affected by these values later on, as the formulas will take weightage of all stat columns.

We print it out to ensure all columns have been rescored.

In [77]:
# Define column indices
scale_columns = list(range(6, 13)) + list(range(14, 20))
reverse_code_column = 13

# Convert columns to numeric, errors='coerce' will turn non-convertible values to NaN
for col_index in scale_columns:
    col_name = players_df_sin.columns[col_index]
    players_df_sin.loc[:, col_name] = pd.to_numeric(players_df_sin[col_name], errors='coerce')

# Convert reverse coding column to numeric
reverse_code_col_name = players_df_sin.columns[reverse_code_column]
players_df_sin.loc[:, reverse_code_col_name] = pd.to_numeric(players_df_sin[reverse_code_col_name], errors='coerce')

# Function to scale values
def scale_values(series):
    min_val = series.min()
    max_val = series.max()
    return 25 + ((series - min_val) / (max_val - min_val)) * 75

# Apply scaling to specified columns
for col_index in scale_columns:
    col_name = players_df_sin.columns[col_index]
    players_df_sin.loc[:, col_name] = scale_values(players_df_sin[col_name])

# Function to reverse code values
def reverse_code_values(series):
    min_val = series.min()
    max_val = series.max()
    return 25 + ((max_val - series) / (max_val - min_val)) * 75

# Apply reverse coding to column 13
players_df_sin.loc[:, reverse_code_col_name] = reverse_code_values(players_df_sin[reverse_code_col_name])

players_df_sin

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,Clean sheets,Saves per game
3,faris ramli,SIN,168 cm,Right,30,M,62.5,87.5,78.25,54.25,54.25,36.746988,48.571429,80.434783,28.125,86.5,61.0,47.5,NaN,NaN
5,shah shahiran,SIN,173 cm,Right,8,M,25.0,50.0,94.0,82.75,47.5,57.53012,73.571429,69.021739,32.8125,93.25,50.5,35.5,NaN,NaN
6,taufik suparno,SIN,165 cm,Right,13,M,50.0,50.0,82.75,100.0,45.25,33.13253,35.714286,93.478261,32.8125,71.5,56.5,68.5,NaN,NaN
7,joel chew joon herng,SIN,168 cm,Right,12,M,25.0,25.0,86.5,25.0,25.0,29.518072,25.0,100.0,32.8125,25.0,25.0,25.0,NaN,NaN
8,yasir hanapi,SIN,170 cm,Right,18,M,37.5,62.5,92.5,87.25,77.5,30.421687,30.714286,95.108696,29.6875,75.25,46.0,67.75,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,faizal roslan,SIN,181 cm,Right,15,D,37.5,25.0,84.25,67.0,29.5,71.084337,80.0,47.826087,64.0625,100.0,52.0,64.75,25.0,NaN
177,syed akmal,SIN,NaN,NaN,5,D,25.0,25.0,89.5,64.0,43.75,62.951807,79.285714,72.282609,57.8125,25.0,48.25,66.25,25.0,NaN
179,fashah iskandar,SIN,187 cm,Right,18,G,25.0,25.0,70.75,52.0,25.0,65.662651,75.0,100.0,40.625,25.0,75.25,100.0,25.0,51.612903
180,prathip ekamparam,SIN,187 cm,Right,1,G,25.0,25.0,88.75,51.25,25.0,49.39759,60.0,95.108696,39.0625,25.0,91.0,100.0,25.0,55.241935


### Rescoring keeper stat columns

Keeper stats are only available for keepers, and as such these columns are empty for all non-keeper players. We fill all of these columns with 0 so we still highlight that outfielders are not fit for filling into these positions, while still retaining the functionality of these above mentioned role scores.

We print it out to ensure all columns have been rescored.

In [80]:
# Define the column indices
columns_to_fill = [18, 19]

# Fill NaNs with 0 in the specified columns
for col_index in columns_to_fill:
    col_name = players_df_sin.columns[col_index]
    # Use .loc[] to ensure we are modifying the DataFrame correctly
    players_df_sin.loc[:, col_name] = players_df_sin[col_name].fillna(0)

players_df_sin

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,Acc. crosses,Interceptions per game,Balls recovered per game,Dribbled past per game,Clearances per game,Succ. dribbles,Total duels won,Aerial duels won,Clean sheets,Saves per game
3,faris ramli,SIN,168 cm,Right,30,M,62.5,87.5,78.25,54.25,54.25,36.746988,48.571429,80.434783,28.125,86.5,61.0,47.5,0.0,0.000000
5,shah shahiran,SIN,173 cm,Right,8,M,25.0,50.0,94.0,82.75,47.5,57.53012,73.571429,69.021739,32.8125,93.25,50.5,35.5,0.0,0.000000
6,taufik suparno,SIN,165 cm,Right,13,M,50.0,50.0,82.75,100.0,45.25,33.13253,35.714286,93.478261,32.8125,71.5,56.5,68.5,0.0,0.000000
7,joel chew joon herng,SIN,168 cm,Right,12,M,25.0,25.0,86.5,25.0,25.0,29.518072,25.0,100.0,32.8125,25.0,25.0,25.0,0.0,0.000000
8,yasir hanapi,SIN,170 cm,Right,18,M,37.5,62.5,92.5,87.25,77.5,30.421687,30.714286,95.108696,29.6875,75.25,46.0,67.75,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,faizal roslan,SIN,181 cm,Right,15,D,37.5,25.0,84.25,67.0,29.5,71.084337,80.0,47.826087,64.0625,100.0,52.0,64.75,25.0,0.000000
177,syed akmal,SIN,NaN,NaN,5,D,25.0,25.0,89.5,64.0,43.75,62.951807,79.285714,72.282609,57.8125,25.0,48.25,66.25,25.0,0.000000
179,fashah iskandar,SIN,187 cm,Right,18,G,25.0,25.0,70.75,52.0,25.0,65.662651,75.0,100.0,40.625,25.0,75.25,100.0,25.0,51.612903
180,prathip ekamparam,SIN,187 cm,Right,1,G,25.0,25.0,88.75,51.25,25.0,49.39759,60.0,95.108696,39.0625,25.0,91.0,100.0,25.0,55.241935


### Generating Role Scores

We weight the scores for each role, with a focus on specific stats for certain roles. For the keeper roles, naturally, clean sheets and save percentages take a much higher weight. Meanwhile, goal poachers have ahigher emphasis on their goalscoring. Each role will take up a new column, adding 11 new columns to the data.

We print it out to ensure the scores have been generated.

In [89]:
import pandas as pd

# Define weights for each role
weights = {
    'Traditional Keeper': [0.01, 0.01, 0.02, 0.02, 0.02, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.24, 0.23],
    'Sweeper Keeper': [0.01, 0.01, 0.1, 0.1, 0.03, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.15, 0.15],
    'Ball-Playing Defender': [0.02, 0.01, 0.15, 0.12, 0.02, 0.12, 0.15, 0.05, 0.1, 0.05, 0.1, 0.11],
    'No-Nonsense Defender': [0.01, 0.01, 0.02, 0.02, 0.02, 0.18, 0.12, 0.12, 0.18, 0.15, 0.1, 0.07],
    'Full-Back': [0.02, 0.05, 0.05, 0.05, 0.15, 0.1, 0.1, 0.05, 0.05, 0.15, 0.15, 0.08],
    'All-Action Midfielder': [0.05, 0.05, 0.08, 0.05, 0.05, 0.15, 0.15, 0.1, 0.08, 0.08, 0.08, 0.08],
    'Midfield Playmaker': [0.02, 0.2, 0.2, 0.08, 0.08, 0.05, 0.05, 0.05, 0.05, 0.1, 0.08, 0.04],
    'Traditional Winger': [0.2, 0.15, 0.05, 0.05, 0.2, 0.02, 0.02, 0.02, 0.05, 0.15, 0.05, 0.04],
    'Inverted Winger': [0.25, 0.15, 0.05, 0.05, 0.05, 0.02, 0.02, 0.02, 0.1, 0.15, 0.15, 0.04],
    'Goal Poacher': [0.35, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.15, 0.15, 0.13],
    'Target Man': [0.2, 0.05, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.05, 0.3, 0.2, 0.08]
}

# Define column indices for each role
col_indices = {
    'Traditional Keeper': list(range(6, 20)),
    'Sweeper Keeper': list(range(6, 20)),
    'Ball-Playing Defender': list(range(6, 18)),
    'No-Nonsense Defender': list(range(6, 18)),
    'Full-Back': list(range(6, 18)),
    'All-Action Midfielder': list(range(6, 18)),
    'Midfield Playmaker': list(range(6, 18)),
    'Traditional Winger': list(range(6, 18)),
    'Inverted Winger': list(range(6, 18)),
    'Goal Poacher': list(range(6, 18)),
    'Target Man': list(range(6, 18))
}

# Make a copy of the DataFrame to avoid SettingWithCopyWarning
players_df_sin_scores = players_df_sin.copy()

# Calculate the score for each role and add it as a new column
for role, weights_list in weights.items():
    if len(weights_list) != len(col_indices[role]):
        raise ValueError(f"Weight list length for '{role}' does not match the number of columns")
    
    weighted_sum = sum(weights_list[i] * players_df_sin_scores.iloc[:, col_index]
                       for i, col_index in enumerate(col_indices[role]))
    players_df_sin_scores[role] = weighted_sum

players_df_sin_scores

,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,...,Sweeper Keeper,Ball Playing Defender,No Nonsense Defender,Full Back,All Action Midfielder,Midfield Playmaker,Traditional Winger,Inverted Winger,Goal Poacher,Target Man
3,faris ramli,SIN,168 cm,Right,30,M,62.5,87.5,78.25,54.25,...,41.24641,55.637092,54.792703,60.272331,57.876241,68.20391,65.746314,68.240064,63.006314,67.281314
5,shah shahiran,SIN,173 cm,Right,8,M,25.0,50.0,94.0,82.75,...,44.759289,64.269166,60.130352,61.566867,61.314906,66.151789,54.413091,55.228716,47.555591,58.543091
6,taufik suparno,SIN,165 cm,Right,13,M,50.0,50.0,82.75,100.0,...,46.464379,60.86571,54.103211,57.30422,56.902348,63.336879,56.864627,59.867752,57.552127,60.177127
7,joel chew joon herng,SIN,168 cm,Right,12,M,25.0,25.0,86.5,25.0,...,28.016529,39.298419,37.449503,32.667432,38.722711,41.666529,30.055986,31.696611,28.210986,28.210986
8,yasir hanapi,SIN,170 cm,Right,18,M,37.5,62.5,92.5,87.25,...,45.734108,59.06693,52.693411,60.448407,56.813765,68.141608,62.269268,58.603643,52.999268,57.574268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,faizal roslan,SIN,181 cm,Right,15,D,37.5,25.0,84.25,67.0,...,50.208646,68.917675,68.638061,62.670363,64.600259,60.218646,52.083833,57.936958,56.388833,65.126333
177,syed akmal,SIN,NaN,NaN,5,D,25.0,25.0,89.5,64.0,...,47.229132,64.934954,57.583274,53.003508,59.396389,54.646632,41.168528,43.571653,40.726028,39.826028
179,fashah iskandar,SIN,187 cm,Right,18,G,25.0,25.0,70.75,52.0,...,57.856318,66.069518,61.861777,55.772515,62.379398,52.394383,38.244503,45.300753,47.837003,46.599503
180,prathip ekamparam,SIN,187 cm,Right,1,G,25.0,25.0,88.75,51.25,...,59.81473,65.651896,58.18586,55.548319,59.738008,55.308439,39.093251,47.646376,49.743251,49.293251


### Generating Recommendation Scores

As the modelling that will take place later on is making use of the pycaret classification system, the existing role scores would not be suitable for the model. As such we code for each of these score columns into a binary recommendation system, taking 50 as the threshold for recommendation or not.

Since the scores are still essential to the end user later on, we retain these columns, and add another 11 columns for the binary recommendations.

We print it out to ensure that the these new columns are now present.

In [90]:
import pandas as pd

# Define the column indices for the original values and the new classification columns
original_columns = list(range(20, 31))  # Columns to classify
classification_columns = list(range(31, 42))  # New columns for classifications

# Function to classify values
def classify_value(value):
    return 1 if value > 50 else 0

# Make a copy of the DataFrame to avoid modifying the original DataFrame
players_df_sin_reco = players_df_sin_scores.copy()

# Apply the classification to each relevant column and create new columns
for orig_col, class_col in zip(original_columns, classification_columns):
    orig_col_name = players_df_sin_reco.columns[orig_col]
    class_col_name = f'Class_{orig_col_name}'
    
    # Apply classification
    players_df_sin_reco[class_col_name] = players_df_sin_reco.iloc[:, orig_col].apply(classify_value)

# Display the updated DataFrame
players_df_sin_reco


,Player Name,NATIONALITY,HEIGHT,PREFERRED FOOT,SHIRT NUMBER,POSITION,Goals per game,Assists,Accurate per game,Acc. long balls,...,Class_Sweeper Keeper,Class_Ball Playing Defender,Class_No Nonsense Defender,Class_Full Back,Class_All Action Midfielder,Class_Midfield Playmaker,Class_Traditional Winger,Class_Inverted Winger,Class_Goal Poacher,Class_Target Man
3,faris ramli,SIN,168 cm,Right,30,M,62.5,87.5,78.25,54.25,...,0,1,1,1,1,1,1,1,1,1
5,shah shahiran,SIN,173 cm,Right,8,M,25.0,50.0,94.0,82.75,...,0,1,1,1,1,1,1,1,0,1
6,taufik suparno,SIN,165 cm,Right,13,M,50.0,50.0,82.75,100.0,...,0,1,1,1,1,1,1,1,1,1
7,joel chew joon herng,SIN,168 cm,Right,12,M,25.0,25.0,86.5,25.0,...,0,0,0,0,0,0,0,0,0,0
8,yasir hanapi,SIN,170 cm,Right,18,M,37.5,62.5,92.5,87.25,...,0,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,faizal roslan,SIN,181 cm,Right,15,D,37.5,25.0,84.25,67.0,...,1,1,1,1,1,1,1,1,1,1
177,syed akmal,SIN,NaN,NaN,5,D,25.0,25.0,89.5,64.0,...,0,1,1,1,1,1,0,0,0,0
179,fashah iskandar,SIN,187 cm,Right,18,G,25.0,25.0,70.75,52.0,...,1,1,1,1,1,1,0,0,0,0
180,prathip ekamparam,SIN,187 cm,Right,1,G,25.0,25.0,88.75,51.25,...,1,1,1,1,1,1,0,0,0,0


With that, the data cleaning and processing is done. We extract the file to CSV, as this file is now the final version that will be used for modelling.

In [69]:
players_df_sin_reco.to_csv('players_df_sin_reco.csv', index=False)

## Next Steps

### Modelling

With this, we begin the modelling; this will be carried out in the next notebook: 02-modelling.ipynb

### EDA 

Alternatively, we can look at the player data in the EDA notebook: 03-eda.ipynb